In [2]:
import pandas as pd
import geopandas as gpd

import folium
import branca

### einlesen

In [3]:
url='https://offenedaten.aachen.de/de/dataset/b621bd10-5f01-4f05-9c2f-317fe0eb48e0/resource/9f1e582e-1b74-4ab1-8b75-bb99133d0c1b/download/pfb11-400it-managementopendatadatenfb32geschwindigkeitskontrollen2020_juni.xlsx'
data=pd.read_excel(url)

data.head()

,Datum,Messplatz,Fahrtrichtung,Fahrzeuge gesamt,bis 15 km/h,über 15 km/h
0,2020-06-02,Horbacher Straße,Roermonder Straße,408,42,1
1,2020-06-02,Neuenhofstraße,Debystraße,348,48,3
2,2020-06-02,Nirmer Straße,Kalkbergstraße,180,22,0
3,2020-06-02,Kesselstraße,Monschauer Straße,147,26,2
4,2020-06-02,Maria Theresia Allee,Luxemburger Ring,61,14,1


### geocoden

In [4]:
data['Adresse']= data.Messplatz + ', Aachen, Germany'

In [6]:
data_locations = gpd.tools.geocode(data.Adresse, provider='nominatim', user_agent="meinekleineapp")
data_locations

,geometry,address
0,POINT (6.05849 50.81519),"Horbacher Straße, Richterich, Aachen, Städtere..."
1,POINT (6.14995 50.76437),"Neuenhofstraße, Gewerbegebiet Eilendorf-Süd, E..."
2,POINT (6.15796 50.78385),"Nirmer Straße, Nirm, Eilendorf, Aachen, Städte..."
3,POINT (6.12894 50.71123),"Kesselstraße, Oberforstbach, Kornelimünster/Wa..."
4,POINT (6.07383 50.75280),"Maria-Theresia-Allee, Steinebrück, Burtscheid,..."
...,...,...
81,POINT (6.14995 50.76437),"Neuenhofstraße, Gewerbegebiet Eilendorf-Süd, E..."
82,POINT (6.10399 50.79733),"Krefelder Straße, Laurensberg, Aachen, Städter..."
83,POINT (6.09516 50.79950),"Soerser Weg, Soers, Aachen, Städteregion Aache..."
84,POINT (6.12894 50.71123),"Kesselstraße, Oberforstbach, Kornelimünster/Wa..."


In [7]:
data_join=data_locations.join(data)
data_join.head()

,geometry,address,Datum,Messplatz,Fahrtrichtung,Fahrzeuge gesamt,bis 15 km/h,über 15 km/h,Adresse
0,POINT (6.05849 50.81519),"Horbacher Straße, Richterich, Aachen, Städtere...",2020-06-02,Horbacher Straße,Roermonder Straße,408,42,1,"Horbacher Straße, Aachen, Germany"
1,POINT (6.14995 50.76437),"Neuenhofstraße, Gewerbegebiet Eilendorf-Süd, E...",2020-06-02,Neuenhofstraße,Debystraße,348,48,3,"Neuenhofstraße, Aachen, Germany"
2,POINT (6.15796 50.78385),"Nirmer Straße, Nirm, Eilendorf, Aachen, Städte...",2020-06-02,Nirmer Straße,Kalkbergstraße,180,22,0,"Nirmer Straße, Aachen, Germany"
3,POINT (6.12894 50.71123),"Kesselstraße, Oberforstbach, Kornelimünster/Wa...",2020-06-02,Kesselstraße,Monschauer Straße,147,26,2,"Kesselstraße, Aachen, Germany"
4,POINT (6.07383 50.75280),"Maria-Theresia-Allee, Steinebrück, Burtscheid,...",2020-06-02,Maria Theresia Allee,Luxemburger Ring,61,14,1,"Maria Theresia Allee, Aachen, Germany"


In [8]:
#data_join.columns

### df aufbereiten
* Anteile
* Wochentag
* Mehrfachnennungen versetzen

In [9]:
data_join['Anteil Verstöße']=(data_join['bis 15 km/h  ']+data_join['über 15 km/h'])/data_join['Fahrzeuge gesamt']
data_join['Anteil Verstöße']=(data_join['Anteil Verstöße']*100).round(1)

In [10]:
data_join['Wochentag']=data_join.Datum.dt.weekday_name
data_join.Datum=data_join.Datum.astype(str)

In [11]:
#versetze location etwas entlang der y-Achse wenn Ort mehrfach vorkommt

In [12]:
data_join['y']=data_join.geometry.y
data_join['x']=data_join.geometry.x

In [13]:
data_join['Messplatz_cumcount']=data_join.groupby(['Messplatz']).cumcount()
data_join.y=data_join.apply(lambda i: i.geometry.y+(i['Messplatz_cumcount']/10000), axis=1)

In [14]:
data_join = gpd.GeoDataFrame(
    data_join, 
    geometry=gpd.points_from_xy(data_join.x, data_join.y),
    crs='EPSG:4326')

In [15]:
data_join

,geometry,address,Datum,Messplatz,Fahrtrichtung,Fahrzeuge gesamt,bis 15 km/h,über 15 km/h,Adresse,Anteil Verstöße,Wochentag,y,x,Messplatz_cumcount
0,POINT (6.05849 50.81519),"Horbacher Straße, Richterich, Aachen, Städtere...",2020-06-02,Horbacher Straße,Roermonder Straße,408,42,1,"Horbacher Straße, Aachen, Germany",10.5,Tuesday,50.815190,6.058488,0
1,POINT (6.14995 50.76437),"Neuenhofstraße, Gewerbegebiet Eilendorf-Süd, E...",2020-06-02,Neuenhofstraße,Debystraße,348,48,3,"Neuenhofstraße, Aachen, Germany",14.7,Tuesday,50.764368,6.149954,0
2,POINT (6.15796 50.78385),"Nirmer Straße, Nirm, Eilendorf, Aachen, Städte...",2020-06-02,Nirmer Straße,Kalkbergstraße,180,22,0,"Nirmer Straße, Aachen, Germany",12.2,Tuesday,50.783848,6.157964,0
3,POINT (6.12894 50.71123),"Kesselstraße, Oberforstbach, Kornelimünster/Wa...",2020-06-02,Kesselstraße,Monschauer Straße,147,26,2,"Kesselstraße, Aachen, Germany",19.0,Tuesday,50.711232,6.128944,0
4,POINT (6.07383 50.75280),"Maria-Theresia-Allee, Steinebrück, Burtscheid,...",2020-06-02,Maria Theresia Allee,Luxemburger Ring,61,14,1,"Maria Theresia Allee, Aachen, Germany",24.6,Tuesday,50.752804,6.073831,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,POINT (6.14995 50.76447),"Neuenhofstraße, Gewerbegebiet Eilendorf-Süd, E...",2020-06-30,Neuenhofstraße,Debystraße,271,10,0,"Neuenhofstraße, Aachen, Germany",3.7,Tuesday,50.764468,6.149954,1
82,POINT (6.10399 50.79733),"Krefelder Straße, Laurensberg, Aachen, Städter...",2020-06-30,Krefelder Straße,Würselen,1115,115,8,"Krefelder Straße, Aachen, Germany",11.0,Tuesday,50.797331,6.103985,0
83,POINT (6.09516 50.79960),"Soerser Weg, Soers, Aachen, Städteregion Aache...",2020-06-30,Soerser Weg,Krefelder Straße,310,52,2,"Soerser Weg, Aachen, Germany",17.4,Tuesday,50.799599,6.095159,1
84,POINT (6.12894 50.71143),"Kesselstraße, Oberforstbach, Kornelimünster/Wa...",2020-06-30,Kesselstraße,Monschauer Straße,237,20,1,"Kesselstraße, Aachen, Germany",8.9,Tuesday,50.711432,6.128944,2


### VIZ

In [21]:
import folium
import branca

#df=data_join[['geometry','Fahrzeuge gesamt']]

df=data_join[['geometry','Datum','Wochentag','Messplatz','Fahrtrichtung','Fahrzeuge gesamt','bis 15 km/h  ','über 15 km/h','Anteil Verstöße']]


##create (base)map
#m = folium.Map(location=[50.75, 6.1], 
#               zoom_start=13, 
#               tiles='cartodbpositron', 
#               attr='My Data Attribution')
#create (base)map

carto_attribution='\u0026copy; \u003ca href=\"http://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"http://cartodb.com/attributions\"\u003eCartoDB\u003c/a\u003e, CartoDB \u003ca href =\"http://cartodb.com/attributions\"\u003eattributions\u003c/a\u003e' # <-- note this

custom_attribution=carto_attribution + ' | \u0026copy; \u003ca href=\"https://offenedaten.aachen.de/de/dataset/geschwindigkeitskontrollen\"\u003eStadt Aachen\u003c/a\u003e'

m = folium.Map(location=[50.75, 6.1], 
               zoom_start=13, 
                tiles='https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png', 
                attr=custom_attribution)


# icon color
def colors(inp):
    if inp < 5:
        col='green'
    elif inp < 10:
        col='orange'
    else:
        col='red'
    return col

#iterrate over ervry row in df to create icon
for ix,row in df.iterrows():
    # build tooltips for icon
    tool_tip_text = '<pre>'
    for p in df.columns:
        if p!='geometry':
            
            
            if p=='Anteil Verstöße':
                tool_tip_text = tool_tip_text + str(p) + ': <b>' + str(row[p]) + '%</b><br>'
            else:
                tool_tip_text = tool_tip_text + str(p) + ': <b>' + str(row[p]) + '</b><br>'
    tool_tip_text = tool_tip_text + '</pre>'
    tool_tip = folium.Tooltip(tool_tip_text)
    
    
    #create marker/icon
    folium.Marker(location=(row['geometry'].y,row['geometry'].x),
                 # popup= row['Messplatz'],
                  tooltip=tool_tip,                 
                  icon=folium.Icon(
                          color=colors(row['Anteil Verstöße']),
                          icon='certificate',
                          #prefix='fa'
                      )
                      ).add_to(m)

    
    
# create custom legend
legend_html =   '''
                <div style="position: fixed; 
                            top: 50px; right: 50px; width: 130px; height: 130px; 
                            z-index:9998; font-size:14px; border-radius: 5px;
                            border:2px solid grey;
                            background-color: white; opacity: 0.9;
                            ">
                </div>
                
                <div style="position: fixed; 
                            top: 50px; right: 50px; width: 130px; height: 150px; padding-top:5px;
                             z-index:9999; font-size:14px;
                        
                            ">&nbsp;<b> Anteil Verstöße </b><br>
                              &nbsp;&nbsp;&nbsp;&nbsp;                > 10 % &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i><br>
                              &nbsp;&nbsp;                          5 - 10 % &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
                              &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;     < 5 % &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>
                </div>
                ''' 

m.get_root().html.add_child(folium.Element(legend_html))    

m

In [200]:
m.save('geschw_aachen_juni2020.html')